In [ ]:
import transformers

In [6]:

import sqlite3

# 데이터베이스 파일 경로
db_path = '/Users/seungbin/study-dev/FinanceSentimentCorpus/datasets/naver_news.db'

# 데이터베이스 연결
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# article_contents 테이블의 ticker 컬럼 값 확인
cursor.execute("SELECT id, ticker FROM article_contents LIMIT 10")
article_contents_ticker = cursor.fetchall()

# articles 테이블의 ticker 컬럼 값 확인
cursor.execute("SELECT id, ticker FROM articles LIMIT 100")
articles_ticker = cursor.fetchall()

conn.close()

article_contents_ticker, articles_ticker


([(1, '211270'),
  (2, '6840'),
  (3, '109960'),
  (4, '139050'),
  (5, '265520'),
  (6, '54620'),
  (7, '211270'),
  (8, '54620'),
  (9, '6840'),
  (10, '60310')],
 [(1, '211270'),
  (2, '54620'),
  (3, '211270'),
  (4, '54620'),
  (5, '265520'),
  (6, '109960'),
  (7, '139050'),
  (8, '6840'),
  (9, '60310'),
  (10, '95570'),
  (11, '211270'),
  (12, '54620'),
  (13, '265520'),
  (14, '109960'),
  (15, '139050'),
  (16, '6840'),
  (17, '60310'),
  (18, '95570'),
  (19, '211270'),
  (20, '54620'),
  (21, '265520'),
  (22, '109960'),
  (23, '139050'),
  (24, '6840'),
  (25, '60310'),
  (26, '95570'),
  (27, '211270'),
  (28, '54620'),
  (29, '265520'),
  (30, '109960'),
  (31, '139050'),
  (32, '6840'),
  (33, '60310'),
  (34, '95570'),
  (35, '211270'),
  (36, '54620'),
  (37, '265520'),
  (38, '109960'),
  (39, '139050'),
  (40, '6840'),
  (41, '60310'),
  (42, '95570'),
  (43, '211270'),
  (44, '54620'),
  (45, '265520'),
  (46, '109960'),
  (47, '139050'),
  (48, '6840'),
  (49, '6

In [23]:
import pandas as pd
from sqlalchemy import create_engine

# Create an engine to connect to the SQLite database
engine = create_engine('sqlite:////Users/seungbin/study-dev/FinanceSentimentCorpus/datasets/naver_news.db')

# Define the query to select the content column from the article_contents table
query = "SELECT * FROM articles"

# Use pandas to read the data from the database
df = pd.read_sql(query, engine)

df.sort_values(by=['media_id'])

# Display the DataFrame
print(df.sort_values(by=['media_name']))

      id  article_id media_id media_name  \
326  327  0011720250      056        KBS   
440  441  0011686779      056        KBS   
582  583  0011667460      056        KBS   
308  309  0011720529      056        KBS   
334  335  0011720084      056        KBS   
..   ...         ...      ...        ...   
202  203  0002292473      016      헤럴드경제   
494  495  0002280984      016      헤럴드경제   
187  188  0002302516      016      헤럴드경제   
264  265  0002282686      016      헤럴드경제   
516  517  0002233843      016      헤럴드경제   

                                      title  \
326                    부산 금융권 ‘부실 PF’ 정리 착수   
440  “퇴직금 부담 낮춘다”…근로복지공단-CU, ‘푸른씨앗’ 확산 업무협약   
582              KT&G 차기 사장 최종후보에 방경만 수석부사장   
308                    부산 금융권 ‘부실 PF’ 정리 착수   
334                    부산 금융권 ‘부실 PF’ 정리 착수   
..                                      ...   
202         BGF 홍정국 부회장, ‘바이바이 플라스틱 챌린지’ 동참   
494      BYC, 20·30대 겨냥 ‘올데이롱’ 출시…“무신사 先공개”   
187       여름에도 쾌적하게…BYC, 기능성 쿨웨어 ‘보디드라이’ 출시   
2

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from sqlalchemy import create_engine

# 데이터베이스에서 데이터 로드
engine = create_engine('sqlite:////content/drive/My Drive/naver_news.db')
query = "SELECT * FROM articles"
df = pd.read_sql(query, engine)
df = df.sort_values(by=['media_name'])

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

# GPU 사용 설정 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 감성 분석을 위한 함수 정의
def analyze_sentiment(text):
    # 텍스트를 토큰화하고 텐서로 변환
    # pytorch 사용하니까 return_tensors: pt
    # 텍스트 토큰화(tokenizer)
    # truncation: 텍스트 길이가 512 초과한 경우 초과부분 잘라냄
    # padding: 텍스트 길이가 최대부분보다 작을 경우 부족한 부분 패딩하여 채움 
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)    
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # 모델을 통해 예측 수행
    # with torch.no_grad()가 적용된 Tensor를 사용하면 계산 시 연산 속도가 빨라진다. 하지만 backpropagation을 할 수 없기에 학습은 불가능하다.
    # 쉽게 생각해서 더이상 학습 안하고 학습된 모델로 결과볼 때 사용하는거라고 이해해도 된다.
    with torch.no_grad():
        # model(**inputs): 모델에 입력 텐서를 전달하여 예측을 수행. 입력은 키워드 인자로 전달되며, 모델은 이에 따라 출력을 생성
        outputs = model(**inputs)
    
    # 예측된 감성 점수 가져오기
    # outputs: 모델의 출력 객체로, 보통 logits 속성을 포함합니다. logits는 각각에 대한 예측 점수입니다.
    logits = outputs.logits
    sentiment = torch.softmax(logits, dim=1)
    
    return sentiment.cpu().numpy()

# 감성 분석 수행 및 결과 저장
results = []
for index, row in df.iterrows():
    text = row['title']
    sentiment = analyze_sentiment(text)
    results.append(sentiment)

# 결과를 데이터프레임에 추가
df['sentiment'] = results

# 감성 예측 결과 sentiment_label에 작성
def get_sentiment_label(sentiment):
    sentiment_map = ['Negative', 'Neutral', 'Positive']
    return sentiment_map[sentiment.argmax()]

df['sentiment_label'] = df['sentiment'].apply(lambda x: get_sentiment_label(x[0]))

# 결과 출력
print(df[['media_name', 'title', 'sentiment', 'sentiment_label']])

# 결과를 CSV 파일로 저장
df.to_csv('sentiment_analysis_results.csv', index=False)


    media_name                                   title  \
326        KBS                    부산 금융권 ‘부실 PF’ 정리 착수   
440        KBS  “퇴직금 부담 낮춘다”…근로복지공단-CU, ‘푸른씨앗’ 확산 업무협약   
582        KBS              KT&G 차기 사장 최종후보에 방경만 수석부사장   
308        KBS                    부산 금융권 ‘부실 PF’ 정리 착수   
334        KBS                    부산 금융권 ‘부실 PF’ 정리 착수   
..         ...                                     ...   
202      헤럴드경제         BGF 홍정국 부회장, ‘바이바이 플라스틱 챌린지’ 동참   
494      헤럴드경제      BYC, 20·30대 겨냥 ‘올데이롱’ 출시…“무신사 先공개”   
187      헤럴드경제       여름에도 쾌적하게…BYC, 기능성 쿨웨어 ‘보디드라이’ 출시   
264      헤럴드경제          LCC 투자한 PE, 주가 흐름에 회수 타이밍 '고심'   
516      헤럴드경제                      블록체인으로 어류 질병 신속 판별   

                                        sentiment sentiment_label  
326       [[0.8135392, 0.18517101, 0.0012897722]]        Negative  
440    [[9.642663e-05, 0.9993181, 0.00058552925]]         Neutral  
582   [[4.5890665e-05, 0.9998622, 9.1897695e-05]]         Neutral  
308       [[0.8135392, 0.185171